In [5]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import kagglehub
from scipy.spatial import distance
from tqdm import tqdm
import os
import sys

# ----------------------------
# Function to partition dataset
# ----------------------------
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

# ----------------------------
# Load and prepare dataset
# ----------------------------
path = kagglehub.dataset_download("emmarex/plantdisease")
base_path = Path(path) / "PlantVillage"
tomato_folders = [f for f in os.listdir(base_path) if f.startswith("Tomato")]

# Load entire dataset
full_dataset = tf.keras.utils.image_dataset_from_directory(
    base_path,
    seed=123,
    shuffle=True,
    image_size=(256, 256),
    batch_size=32,
    class_names=tomato_folders
)

_, val_ds, _ = get_dataset_partitions_tf(full_dataset, train_split=0.8, val_split=0.1, test_split=0.1)

# ----------------------------
# Load trained model
# ----------------------------
sys.path.append(os.path.join(os.path.abspath('..')))
disease_model = tf.keras.models.load_model("../models/tomato_main.keras")

# Force model to be built so we can access input/output tensors
_ = disease_model(tf.zeros((1, 256, 256, 3)))

# Extract second-to-last layer outputs
feature_extractor = tf.keras.Model(
    inputs=disease_model.input,
    outputs=disease_model.layers[-2].output
)

# ----------------------------
# Extract features and MSP values
# ----------------------------
features = []
msp_values = []

for images, _ in tqdm(val_ds, desc="Extracting features"):
    preds = disease_model(images, training=False)
    feature_vecs = feature_extractor(images, training=False)

    features.append(feature_vecs.numpy())
    msp_values.extend(np.max(preds.numpy(), axis=1))

features = np.concatenate(features, axis=0)
msp_values = np.array(msp_values)

# ----------------------------
# Compute OOD thresholds
# ----------------------------
mean_feature = np.mean(features, axis=0)
cov_feature = np.cov(features.T)

mahala_distances = [
    distance.mahalanobis(f, mean_feature, np.linalg.inv(cov_feature)) for f in features
]

msp_threshold = np.percentile(msp_values, 5)            # Bottom 5% MSP = likely OOD
mahala_threshold = np.percentile(mahala_distances, 95)  # Top 5% distance = likely OOD

# ----------------------------
# Save to disk for API use
# ----------------------------
os.makedirs("models", exist_ok=True)
np.save("models/msp_threshold.npy", msp_threshold)
np.save("models/mahala_threshold.npy", mahala_threshold)
np.save("models/mean_feature.npy", mean_feature)
np.save("models/cov_feature.npy", cov_feature)

# ----------------------------
# Print for reference
# ----------------------------
print(f"MSP Threshold: {msp_threshold:.4f}")
print(f"Mahalanobis Threshold: {mahala_threshold:.4f}")


Found 16011 files belonging to 10 classes.


AttributeError: The layer sequential_1 has never been called and thus has no defined input.